# Installing all needed dataframes

In [5]:
!pip install nba_api matplotlib seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [6]:
from nba_api.stats.endpoints import BoxScoreAdvancedV2, LeagueDashTeamStats, LeagueGameFinder, LeagueDashPlayerStats, PlayByPlayV2, LeagueLineupViz, TeamPlayerOnOffDetails, TeamPlayerOnOffSummary
import pandas as pd
from tqdm import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [ ]:
season = "2023-24"
game_id = "0022300881"
team_id = "1610612741"

In [1]:
team_stats = pd.read_csv("..data/team_stats.csv")
player_stats = pd.read_csv("..data/player_stats.csv")
lineup_data = pd.read_csv("..data/lineup_data.csv")
game_stats = pd.read_csv("..data/game_stats.csv")
all_team_on_off_details = pd.read_csv("..data/team_on_off_details.csv")
all_player_on_details = pd.read_csv("..data/player_on_details.csv")
all_player_off_details = pd.read_csv("..data/player_off_details.csv")
all_events = pd.read_csv("..data/events.csv")

NameError: name 'pd' is not defined

In [ ]:
all_player_on_details[all_player_on_details['TEAM_ID'] == 1610612744]

In [ ]:
game = game_stats.loc[game_stats['TEAM_ID'] == 1610612744].reset_index(drop=True)
game.loc[0]

In [ ]:

def plot_on_off_rating_difference(players_on_details, player_off_details, team_id, rating='NET_RATING'):
  """
  Plots the difference in on/off ratings for each player in a given team.

  Args:
      players_on_details: DataFrame of player on-court details.
      player_off_details: DataFrame of player off-court details.
      team_id: ID of the team to plot.
      rating: Rating to use
      (default is 'NET_RATING')
  """

  players_on_details = players_on_details.loc[players_on_details['TEAM_ID'] == team_id]
  player_off_details = player_off_details.loc[player_off_details['TEAM_ID'] == team_id]

  # Merge the dataframes
  player_on_off_details = pd.merge(players_on_details, player_off_details, on='VS_PLAYER_ID', suffixes=('_on', '_off'))

  # Calculate the difference in rating
  player_on_off_details['RATING_DIFF'] = player_on_off_details[f'{rating}_on'] - player_on_off_details[f'{rating}_off']

  # Create the plot
  plt.figure(figsize=(12, 8))
  sns.barplot(x='VS_PLAYER_NAME_on', y='RATING_DIFF', data=player_on_off_details)
  plt.xticks(rotation=90)
  plt.xlabel('Player Name')
  plt.ylabel('On/Off Rating Difference')
  plt.title(f'Difference in On/Off {rating.upper()} for Each Player in ')
  plt.tight_layout()
  plt.show()

In [ ]:
plot_on_off_rating_difference(player_on_details, player_off_details, team_id=team_id, rating='NET_RATING')

In [ ]:
def plot_team_rating_over_substitutions(substitution_events, player_on_details, player_off_details, team_on_off_details, game_id):
    """
    Plots the team's rating over time, updating it at each substitution.

    Args:
        substitution_events: DataFrame of substitution events.
        player_on_details: DataFrame of player on-court details.
        player_off_details: DataFrame of player off-court details.
    """

    # Placeholder for team rating (replace with your actual rating calculation)

    substitution_events = substitution_events.loc[substitution_events['GAME_ID'] == game_id]

    team_ids = substitution_events['PLAYER1_TEAM_ID'].unique()
    team_names = substitution_events['PLAYER1_TEAM_NAME'].unique()
    substitution_times_list = []
    team_ratings_list = []

    for i, team_id in enumerate(team_ids):
        team_on_off_details = team_on_off_details.loc[team_on_off_details['TEAM_ID'] == team_id]
        player_on_details = player_on_details.loc[player_on_details['TEAM_ID'] == team_id]
        player_off_details = player_off_details.loc[player_off_details['TEAM_ID'] == team_id]

        team_rating = team_on_off_details['NET_RATING'].values[0]
        team_ratings_over_time = []
        substitution_times = []

        for index, row in substitution_events.iterrows():
            substitution_times.append(row['PCTIMESTRING'])

            incoming_player = row['PLAYER1_ID']
            outgoing_player = row['PLAYER2_ID']

            incoming_player_rating = player_on_details[player_on_details['VS_PLAYER_ID'] == incoming_player]['NET_RATING'].values[0]
            outgoing_player_rating = player_off_details[player_off_details['VS_PLAYER_ID'] == outgoing_player]['NET_RATING'].values[0]

            team_rating = team_rating + incoming_player_rating - outgoing_player_rating

            team_ratings_over_time.append(team_rating)
        substitution_times_list.append(substitution_times)
        team_ratings_list.append(team_ratings_over_time)

    fig, (ax1, ax2) = plt.subplots(2)

    # Create the plot
    fig.suptitle('Team Rating Over Time (Substitutions)')
    ax1.plot(substitution_times_list[0], team_ratings_list[0], marker='o')
    ax1.set_xlabel('Time')
    ax1.set_ylabel(f'{team_names[0]} Rating')
    ax1.set_title(f'{team_names[0]} Rating Over Time (Substitutions)')

    ax2.plot(substitution_times_list[1], team_ratings_list[1], marker='o')
    ax2.set_xlabel('Time')
    ax2.set_ylabel(f'{team_names[1]} Rating')
    ax2.set_title(f'{team_names[1]} Rating Over Time (Substitutions)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()